# Análisis Exploratorio de Datos: Tiempos de Inactividad en Manufactura

Este notebook realiza un análisis exploratorio inicial de los datos de tiempo de inactividad en la línea de producción.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sys
sys.path.append('..')

from utils.db_connection import create_connection, read_table
from utils.visualization import plot_downtime_distribution, plot_operator_performance, create_downtime_heatmap
from utils.statistics import calculate_basic_stats, analyze_operator_performance

%matplotlib inline
plt.style.use('seaborn')

## 1. Carga de Datos

Conectamos a la base de datos y cargamos las tablas necesarias.

In [ ]:
# Crear conexión a la base de datos
conn = create_connection()

# Cargar datos de producción
production_data = read_table(conn, 'manufacturing_line_productivity')
downtime_data = read_table(conn, 'normalizeddowntime')
factors_data = read_table(conn, 'downtime_factor')

# Mostrar las primeras filas de cada tabla
print("\nDatos de Producción:")
print(production_data.head())
print("\nDatos de Tiempo de Inactividad:")
print(downtime_data.head())
print("\nFactores de Tiempo de Inactividad:")
print(factors_data.head())

## 2. Análisis de Distribución de Tiempos de Inactividad

In [ ]:
# Visualizar distribución de tiempos de inactividad
plot_downtime_distribution(downtime_data)

# Estadísticas básicas
stats = calculate_basic_stats(downtime_data['downtime_minutes'])
print("\nEstadísticas de Tiempo de Inactividad (minutos):")
print(f"Media: {stats['mean']:.2f}")
print(f"Mediana: {stats['median']:.2f}")
print(f"Desviación Estándar: {stats['std']:.2f}")
print(f"Mínimo: {stats['min']:.2f}")
print(f"Máximo: {stats['max']:.2f}")

## 3. Análisis por Operador

In [ ]:
# Visualizar rendimiento por operador
plot_operator_performance(production_data, downtime_data)

# Análisis detallado de rendimiento por operador
operator_stats = analyze_operator_performance(production_data, downtime_data)
print("\nEstadísticas por Operador:")
print(operator_stats)

## 4. Análisis de Factores de Tiempo de Inactividad

In [ ]:
# Unir datos de tiempo de inactividad con factores
downtime_with_factors = pd.merge(downtime_data, factors_data, on='factor')

# Crear gráfico de Pareto
factor_counts = downtime_with_factors.groupby('description')['downtime_minutes'].sum().sort_values(ascending=False)
total_downtime = factor_counts.sum()
cumulative_percentage = factor_counts.cumsum() / total_downtime * 100

fig, ax1 = plt.subplots(figsize=(12, 6))
ax2 = ax1.twinx()

ax1.bar(range(len(factor_counts)), factor_counts, color='b', alpha=0.6)
ax2.plot(range(len(factor_counts)), cumulative_percentage, color='r', marker='o')

ax1.set_xlabel('Factores')
ax1.set_ylabel('Tiempo Total de Inactividad (min)')
ax2.set_ylabel('Porcentaje Acumulado')

plt.title('Análisis de Pareto: Factores de Tiempo de Inactividad')
plt.xticks(range(len(factor_counts)), factor_counts.index, rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../visualizations/downtime_factors_pareto.png')
plt.show()

## 5. Análisis de Correlaciones

In [ ]:
# Crear mapa de calor de correlaciones
create_downtime_heatmap(production_data, downtime_data)

# Análisis de correlación entre tamaño del producto y tiempo de inactividad
plt.figure(figsize=(10, 6))
sns.scatterplot(data=production_data.merge(downtime_data.groupby('batch')['downtime_minutes'].sum().reset_index()),
                x='size', y='downtime_minutes')
plt.title('Correlación entre Tamaño del Producto y Tiempo de Inactividad')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../visualizations/size_correlation.png')
plt.show()

## 6. Conclusiones Preliminares

1. Distribución de Tiempos de Inactividad:
   - [Completar con observaciones sobre la distribución]

2. Rendimiento por Operador:
   - [Completar con hallazgos sobre operadores]

3. Factores Principales:
   - [Completar con análisis de factores más significativos]

4. Correlaciones Identificadas:
   - [Completar con patrones y correlaciones encontradas]

## Próximos Pasos

1. Realizar análisis más detallado de los factores principales
2. Investigar patrones temporales en los datos
3. Desarrollar modelos predictivos
4. Generar recomendaciones específicas para mejora